# Fornax MRD + CCSNe r-process (2-Zone OMEGA+)
This notebook explores adding core-collapse supernova (CCSNe) r-process enrichment to the MRD channel to explain the observed [Eu/Fe] in Fornax. CCSNe have much higher rates (~0.01-0.1 events/Msun) than NSM (~1e-5 to 1e-4), allowing lower yields per event to achieve the required enrichment.

In [1]:
# Import the OMEGA+ code and standard packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import re
from pathlib import Path
from functools import partial

# make sure the parent folder (which contains JINAPyCEE and NUPYCEE) is on sys.path
sys.path.insert(0, str(Path.cwd().parent))
from JINAPyCEE import omega_plus

plt.style.use('seaborn-v0_8-muted')
plt.rcParams['figure.figsize'] = (10, 7)
plt.rcParams['axes.grid'] = True

# Resolve paths relative to notebook location
repo_root = Path('.').resolve()
package_root = repo_root.parent

## Load Reichert et al. (2020) Fornax Observations

In [2]:
reichert_csv = repo_root / "observations" / "reichert2020_for.csv"

obs_df = pd.read_csv(reichert_csv)
obs_df = obs_df.rename(columns={"ID": "star"})
obs_df = obs_df.dropna(subset=["[Fe/H]", "[Eu/Fe]"])
obs_df = obs_df.sort_values("[Fe/H]").reset_index(drop=True)

print(f"Loaded {len(obs_df)} Reichert et al. (2020) Fornax stars with Eu measurements")
print(f"[Fe/H] range: {obs_df['[Fe/H]'].min():.2f} to {obs_df['[Fe/H]'].max():.2f}")
print(f"[Eu/Fe] range: {obs_df['[Eu/Fe]'].min():.2f} to {obs_df['[Eu/Fe]'].max():.2f}")
obs_df[["star", "[Fe/H]", "[Eu/Fe]", "sigma_Eu", "sigma_Fe"]].head(10)

Loaded 108 Reichert et al. (2020) Fornax stars with Eu measurements
[Fe/H] range: -1.64 to -0.31
[Eu/Fe] range: -0.36 to 1.45


,star,[Fe/H],[Eu/Fe],sigma_Eu,sigma_Fe
0,[LDH2014] Fnx-mem0654,-1.64,0.31,0.15,0.06
1,[LDH2014] Fnx-mem0612,-1.54,0.48,0.14,0.07
2,[LHT2010] BL147,-1.50,0.98,0.31,0.05
3,[LDH2014] Fnx-mem0747,-1.35,0.69,0.10,0.06
4,[LDH2014] Fnx-mem0647,-1.34,0.19,0.07,0.07
5,[LDH2014] Fnx-mem0546,-1.33,1.45,0.19,0.08
6,2MASS J02381393-3446553,-1.32,0.29,0.15,0.04
7,[WMO2009] For-0970,-1.23,-0.21,0.22,0.05
8,2MASS J02401043-3425177,-1.01,0.43,0.10,0.10
9,[WMO2009] For-0361,-1.01,0.48,0.08,0.11


## R-Process Yield Tables
We use two complementary r-process sources from NuPyCEE:
- **MRD channel**: Nishimura et al. (2017) L1.00 yields (prompt, high r-process yields)
- **CCSNe channel**: Arnould et al. (2007) general r-process table (lower yields per event, but 100-1000× higher rate from massive stars)

In [ ]:
def format_isotope(label: str) -> tuple[str, int]:
    """Convert labels like 'eu151' to ('Eu-151', 151)."""
    match = re.fullmatch(r"([a-zA-Z]+)([0-9]+)", label.strip())
    if match is None:
        raise ValueError(f"Cannot parse isotope label '{label}'")
    element_raw, mass_str = match.groups()
    element = element_raw.capitalize()
    if len(element_raw) > 1:
        element = element_raw[0].upper() + element_raw[1:].lower()
    mass_number = int(mass_str)
    return f"{element}-{mass_number}", mass_number


# Available r-process yield tables in NuPyCEE:
# 1. Rosswog et al. 2014 - NSM (neutron star merger) r-process
# 2. Arnould et al. 2007 - General r-process reference
# 3. MRD yields (Nishimura et al. 2017) in additional_sources/

# For this analysis, we use Arnould et al. 2007 as a credible r-process baseline
# and interpret different DTDs as different physical channels (MRD vs prompt CCSNe)

mrd_source = repo_root / "yield_tables" / "additional_sources" / "MRD_r_process" / "L1.00.dat"
arnould_source = repo_root / "yield_tables" / "r_process_arnould_2007.txt"
rosswog_source = repo_root / "yield_tables" / "r_process_rosswog_2014.txt"

print("Available r-process yield tables:")
print(f"  - Rosswog et al. 2014 (NSM): {rosswog_source.exists()}")
print(f"  - Arnould et al. 2007 (r-process): {arnould_source.exists()}")
print(f"  - MRD yields (Nishimura et al. 2017): {mrd_source.exists()}")
print(f"\nUsing Arnould et al. 2007 for CCSNe r-process channel.")


## DTD Builders for Prompt MRD and CCSNe r-process

In [ ]:
def build_prompt_dtd(
    metallicities: list[float],
    prompt_window: tuple[float, float] = (3.0e6, 3.0e8),
    t_end: float = 12.0e9,
) -> list[list]:
    """
    Build a simple "prompt" Delay-Time Distribution (DTD) curve for MRD enrichment.
    """
    t_start, t_stop = prompt_window
    if t_stop <= t_start:
        raise ValueError("prompt_window must span a positive duration")

    base_times = [
        0.0,
        t_start,
        t_start * 1.001,
        t_stop,
        t_stop * 1.001,
        t_end,
        t_end + 1.0,  # Dummy point to avoid index out of bounds
    ]
    base_rates = [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]

    return [[base_times[:], base_rates[:], Z] for Z in metallicities]


def build_ccsne_dtd(
    metallicities: list[float],
    t_end: float = 12.0e9,
) -> list[list]:
    """
    Build a prompt DTD for CCSNe r-process (happens ~1-10 Myr after star formation).
    CCSNe have very short delay times since they come from massive stars.
    """
    # CCSNe r-process: brief enrichment window at very early times
    base_times = [
        0.0,
        1.0e6,    # Start at 1 Myr
        1.001e6,  # Sharp rise
        1.0e7,    # End at 10 Myr (tail off as massive stars die)
        1.001e7,  # Sharp drop
        t_end,
        t_end + 1.0,  # Dummy point
    ]
    base_rates = [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]

    return [[base_times[:], base_rates[:], Z] for Z in metallicities]

## Helper Functions for Track Extraction and Fitting

In [ ]:
def extract_track(model: omega_plus.omega_plus, zone: int = 0, xaxis: str = '[Fe/H]', yaxis: str = '[Eu/Fe]') -> pd.DataFrame:
    """Extract a model spectroscopic track from a specified zone."""
    if zone != 0:
        print(f"Warning: OMEGA+ only provides a full inner zone omega instance. "
              f"Requested zone={zone} will fall back to inner zone.")
    
    zone_model = model.inner
    
    x_vals, y_vals = zone_model.plot_spectro(
        xaxis=xaxis,
        yaxis=yaxis,
        solar_norm='Lodders_et_al_2009',
        return_x_y=True,
    )
    
    track = pd.DataFrame({xaxis: x_vals, yaxis: y_vals})
    track = track.replace([-np.inf, np.inf], np.nan).dropna().drop_duplicates(subset=[xaxis])
    return track.sort_values(xaxis).reset_index(drop=True)


def interpolate_model(track: pd.DataFrame, targets: np.ndarray, xaxis: str, yaxis: str) -> np.ndarray:
    """Linear interpolation of model track to observed [Fe/H] values."""
    if track.empty:
        return np.full_like(targets, np.nan, dtype=float)
    x_model = track[xaxis].to_numpy()
    y_model = track[yaxis].to_numpy()
    if len(x_model) < 2:
        return np.full_like(targets, np.nan, dtype=float)
    return np.interp(targets, x_model, y_model, left=np.nan, right=np.nan)

## Baseline Model (No MRD or CCSNe)

In [ ]:
# GCE control parameters for Fornax
fornax_control = {
    "sfr_efficiency_per_gyr": 0.03,
    "tau_infall_gyr": 10.0,
    "tau_star_gyr": 25.0,
    "tau_outflow_gyr": 10.0,
    "wind_epoch_gyr": 12.0,
    "inflow_cutoff_gyr": 9.0,
    "target_gas_mass": 1.0e4,
    "inflow_norm_mass": 1.0e4,
    "t_end_gyr": 12.0,
}

baseline_kwargs = dict(
    ns_merger_on=False,
    imf_yields_range=[1.0, 40.0],
    table="yield_tables/agb_and_massive_stars_C15_LC18_R_mix.txt",
    iolevel=0,
    special_timesteps=100,
)

print("Running baseline 2-zone OMEGA+ model for Fornax...")
baseline_model = omega_plus.omega_plus(**baseline_kwargs)
baseline_inner = extract_track(baseline_model, zone=0)

print(f"Inner zone: {len(baseline_inner)} points")

## MRD + CCSNe Combined Model
Test combining prompt MRD with CCSNe r-process at different rate ratios.

In [ ]:
def evaluate_channel(
    rate: float,
    dtd_builder,
    metallicities: list[float],
    yield_path: Path,
    obs: pd.DataFrame,
    channel_name: str = "r-process",
) -> dict:
    """
    Run a 2-zone OMEGA+ model with a specific r-process channel and DTD.
    """
    try:
        # Build DTD for this channel
        dtd = dtd_builder(metallicities)
        
        # Validate DTD structure
        if not dtd or len(dtd[0]) != 3:
            raise ValueError(f"DTD structure invalid: expected [times, rates, Z] but got {dtd[0] if dtd else 'empty'}")
        
        for i_z, z_entry in enumerate(dtd):
            if len(z_entry[0]) != len(z_entry[1]):
                raise ValueError(f"DTD metallicity {i_z}: times ({len(z_entry[0])}) != rates ({len(z_entry[1])})")
        
        # Prepare OMEGA+ kwargs
        base_kwargs = dict(
            ns_merger_on=True,
            imf_yields_range=[1.0, 40.0],
            table="yield_tables/agb_and_massive_stars_C15_LC18_R_mix.txt",
            iolevel=0,
            special_timesteps=50,
            nb_nsm_per_m=rate,
            nsmerger_dtd_array=dtd,
            nsmerger_table=str(yield_path),
        )
        
        print(f"\nEvaluating {channel_name}: rate={rate:.2e} events/Msun, yield_table={yield_path.name}")
        
        model = omega_plus.omega_plus(**base_kwargs)
        track = extract_track(model, zone=0)
        
        if track.empty:
            return {
                'channel': channel_name,
                'rate': rate,
                'rms': np.nan,
                'track': track,
                'model': model,
            }
        
        # Interpolate to observed [Fe/H]
        interp = interpolate_model(track, obs['[Fe/H]'].to_numpy(), '[Fe/H]', '[Eu/Fe]')
        residuals = obs['[Eu/Fe]'].to_numpy() - interp
        residuals = residuals[np.isfinite(residuals)]
        rms = np.sqrt(np.mean(residuals**2)) if residuals.size else np.nan
        
        return {
            'channel': channel_name,
            'rate': rate,
            'rms': rms,
            'track': track,
            'model': model,
        }
    except Exception as e:
        print(f"Error in {channel_name} evaluation: {e}")
        import traceback
        traceback.print_exc()
        return {
            'channel': channel_name,
            'rate': rate,
            'rms': np.nan,
            'track': pd.DataFrame(),
            'model': None,
        }


## Channel Comparison: MRD vs CCSNe
Scan rates for both r-process channels using their respective yield tables and DTDs.

In [ ]:
# Scan rates for each channel separately
print("=" * 60)
print("CHANNEL SCAN: MRD (Nishimura et al. 2017)")
print("=" * 60)

mrd_rates = np.logspace(-4, -2, 5)  # [1e-4, 1e-3, 1e-2, ...] events/Msun
print(f"MRD rates to scan: {mrd_rates}")

mrd_builder = partial(build_prompt_dtd, prompt_window=(1.0e7, 1.0e8))
mrd_results = [
    evaluate_channel(
        rate,
        mrd_builder,
        metallicity_grid,
        mrd_source,
        obs_df,
        channel_name="MRD (Nishimura et al. 2017)",
    )
    for rate in mrd_rates
]

print("\n" + "=" * 60)
print("CHANNEL SCAN: CCSNe (Arnould et al. 2007)")
print("=" * 60)

ccsn_rates = np.logspace(-2, -1, 5)  # [0.01, 0.032, 0.1, ...] events/Msun
print(f"CCSNe rates to scan: {ccsn_rates}")

ccsn_builder = partial(build_ccsne_dtd)
ccsn_results = [
    evaluate_channel(
        rate,
        ccsn_builder,
        metallicity_grid,
        arnould_source,
        obs_df,
        channel_name="CCSNe (Arnould et al. 2007)",
    )
    for rate in ccsn_rates
]

print(f"\nMRD scan complete: {len(mrd_results)} models")
print(f"CCSNe scan complete: {len(ccsn_results)} models")


## Results Summary

In [ ]:
# Create summary dataframes
mrd_summary = pd.DataFrame({
    'Rate [events/Msun]': [res['rate'] for res in mrd_results],
    'RMS [dex]': [res['rms'] for res in mrd_results],
})

ccsn_summary = pd.DataFrame({
    'Rate [events/Msun]': [res['rate'] for res in ccsn_results],
    'RMS [dex]': [res['rms'] for res in ccsn_results],
})

print("\n" + "=" * 60)
print("MRD Scan Results (Nishimura et al. 2017)")
print("=" * 60)
print(mrd_summary)

print("\n" + "=" * 60)
print("CCSNe Scan Results (Arnould et al. 2007)")
print("=" * 60)
print(ccsn_summary)

# Find best fits for each channel
mrd_best = min(mrd_results, key=lambda res: res['rms'] if np.isfinite(res['rms']) else np.inf)
ccsn_best = min(ccsn_results, key=lambda res: res['rms'] if np.isfinite(res['rms']) else np.inf)

print("\n" + "=" * 60)
print("BEST-FIT COMPARISON")
print("=" * 60)
print(f"\nMRD (Nishimura et al. 2017):")
print(f"  Rate: {mrd_best['rate']:.2e} events/Msun")
print(f"  RMS: {mrd_best['rms']:.4f} dex")

print(f"\nCCSNe (Arnould et al. 2007):")
print(f"  Rate: {ccsn_best['rate']:.2e} events/Msun")
print(f"  RMS: {ccsn_best['rms']:.4f} dex")

if np.isfinite(mrd_best['rms']) and np.isfinite(ccsn_best['rms']):
    delta = mrd_best['rms'] - ccsn_best['rms']
    better = "CCSNe" if delta > 0 else "MRD"
    print(f"\nBetter fit: {better} (Δ RMS = {abs(delta):.4f} dex)")


## Visualization: Best-Fit Track

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for ax_idx, (ax, best_result, channel_name, yield_source) in enumerate([
    (axes[0], mrd_best, "MRD", "Nishimura et al. 2017"),
    (axes[1], ccsn_best, "CCSNe", "Arnould et al. 2007"),
]):
    # Observations
    ax.scatter(obs_df['[Fe/H]'], obs_df['[Eu/Fe]'], s=50, c='black', alpha=0.6, label='Reichert et al. 2020', zorder=3)
    
    # Baseline
    if not baseline_inner.empty:
        ax.plot(baseline_inner['[Fe/H]'], baseline_inner['[Eu/Fe]'], color='gray', linewidth=2, label='Baseline (no r-process)', linestyle=':', zorder=1)
    
    # Best-fit track
    if best_result['model'] is not None and not best_result['track'].empty:
        color = 'tomato' if channel_name == 'MRD' else 'navy'
        ax.plot(best_result['track']['[Fe/H]'], best_result['track']['[Eu/Fe]'], 
                color=color, linewidth=2.5, 
                label=f"{channel_name} best-fit (RMS={best_result['rms']:.3f} dex)", zorder=2)
    
    ax.set_xlabel('[Fe/H]', fontsize=12)
    ax.set_ylabel('[Eu/Fe]', fontsize=12)
    ax.set_title(f'{channel_name} r-process ({yield_source})\nRate = {best_result["rate"]:.2e} events/Msun', 
                 fontsize=12, fontweight='bold')
    ax.legend(fontsize=10, loc='best')
    ax.grid(True, alpha=0.3)
    ax.set_xlim([-2.0, 0.5])
    ax.set_ylim([-2.0, 2.0])

fig.suptitle('MRD vs CCSNe r-process Channels', fontsize=14, fontweight='bold', y=1.02)
fig.tight_layout()
plt.show()


## Yield Table Sources and References

### Nishimura et al. (2017) - MRD r-process
- **Source**: Nishimura et al. 2017, ApJ 836, L21
- **Channel**: Magnetorotational-driven r-process from neutron star mergers
- **Available grids**: L0.10 - L1.25 (L=luminosity parameter)
- **Characteristic rate**: 1e-5 to 1e-4 events/Msun
- **DTD timescale**: Prompt + extended (few Myr to Gyr)

### Arnould et al. (2007) - General r-process
- **Source**: Arnould et al. 2007, Physics Reports 450, 97
- **Channel**: Canonical r-process (endpoint conditions)
- **Coverage**: Full r-process pattern from Zn to Bi
- **Rate interpretation**: Can be applied to any r-process source with adjustment
- **For CCSNe**: Represents lower-yield-per-event r-process from massive star core collapses

### Physical Interpretation
- **MRD**: High r-process yield per event (~0.1-1% of ejecta mass), rare events (NSM rates)
- **CCSNe**: Lower r-process yield per event (~0.01-0.1% of ejecta mass), common events (high CCSN rates ~0.01-0.1 events/Msun)
- **Combined effect**: Early, abundant CCSNe enrichment (1-10 Myr) + rare, r-process-rich NSM events (prompt to Gyr timescale)

### Key Assumptions
1. Both channels produce r-process nuclei via the same general mechanism (rapid neutron capture)
2. Arnould et al. yields are flexible enough to represent CCSNe r-process with appropriate rate scaling
3. DTD shape (early vs late) distinguishes physical channel rather than yield table alone
